In [1]:
!pip install faker
!pip install random

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [2]:
import pandas as pd
from faker import Faker
from sqlalchemy import create_engine
import mysql.connector
from mysql.connector import Error
import random
import numpy as np

In [3]:
# Database credentials
db_params = {
    "host": "localhost",
    "user": "root",
    "passwd": "YOURPASSWORD",
    "database": "new"
}

# Initialize Faker for generating fake data
fake = Faker()

num_records = 10


In [4]:
# Define a function to generate fake data
def generate_fake_data(num_records):
    data = {
        'name': [fake.name() for _ in range(num_records)],
        'email': [fake.email() for _ in range(num_records)],
        'address': [fake.address() for _ in range(num_records)],
        'purchase_amount': [round(fake.random_number(digits=2) + fake.random_digit()/100, 2) for _ in range(num_records)],
        'purchase_date': [fake.date_between(start_date='-1y', end_date='today').isoformat() for _ in range(num_records)]
    }
    return pd.DataFrame(data)

# Extract data from MySQL
def extract_from_mysql(db_params, table_name):
    try:
        connection = mysql.connector.connect(**db_params)
        query = f"SELECT * FROM {table_name}"
        df_mysql = pd.read_sql(query, connection)
        return df_mysql
    except Error as error:
        print(f"Error: {error}")
    finally:
        if connection.is_connected():
            connection.close()

# Transform phase (clean and format data)
def transform(dataframe):
    dataframe['email'] = dataframe['email'].str.lower()
    dataframe['purchase_date'] = pd.to_datetime(dataframe['purchase_date']).dt.strftime('%Y-%m-%d')

    return dataframe

# Load data to MySQL
def load_to_mysql(dataframe, table_name, db_params):
    engine_str = f"mysql+pymysql://{db_params['user']}:{db_params['passwd']}@{db_params['host']}/{db_params['database']}"
    engine = create_engine(engine_str)
    dataframe.to_sql(table_name, engine, if_exists='replace', index=False)

# ETL process function
def etl_process():
    # Extract
    df_mysql = extract_from_mysql(db_params, 'data_from_sql')
    
    # Generate fake data
    df_fake = generate_fake_data(num_records)  # Generate fake records
    
    # Get the last id from the MySQL extracted data
    last_id = df_mysql['id'].max()

    # Assign new incremental ids for the fake data
    df_fake['id'] = range(last_id + 1, last_id + 1 + len(df_fake))

    # Transform
    df_mysql = transform(df_mysql)
    df_fake = transform(df_fake)
    
    # Combine data
    df_combined = pd.concat([df_mysql, df_fake], ignore_index=True)
    
    # Reorder the columns to have 'id' first
    df_combined = df_combined[['id', 'name', 'email', 'address', 'purchase_amount', 'purchase_date']]
    
    # Load to MySQL
    load_to_mysql(df_combined, 'new_table', db_params)

    # Save the combined data to a new CSV file
    output_file = 'combined_data.csv'
    df_combined.to_csv(output_file, index=False)
    
    print(f"ETL process completed and data loaded to MySQL database successfully! Combined data is saved to {output_file}")

# Execute the ETL process
etl_process()


C:\Users\randy\AppData\Local\Temp\ipykernel_12252\3711961086.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mysql = pd.read_sql(query, connection)


ETL process completed and data loaded to MySQL database successfully! Combined data is saved to combined_data.csv


In [5]:
import numpy as np
import random
import pandas as pd

# Additional transformations on the DataFrame
def additional_transformations(dataframe):
    # Randomly set 10% of email column to NaN
    dataframe.loc[np.random.choice(dataframe.index, size=int(0.1 * len(dataframe)), replace=False), 'email'] = np.nan

    # Change format for 50% of purchase_date from YYYY-MM-DD to YYYY/MM/DD
    dataframe.loc[dataframe.sample(frac=0.50).index, 'purchase_date'] = \
        dataframe['purchase_date'].apply(lambda x: x.replace('-', '/'))

    # Convert names to lowercase for 5% of the records
    dataframe.loc[dataframe.sample(frac=0.50).index, 'name'] = \
        dataframe['name'].str.lower()

    # Convert emails to uppercase for 5% of the records
    dataframe.loc[dataframe.sample(frac=0.05).index, 'email'] = \
        dataframe['email'].str.upper()

    # Introduce duplicates for 5% of the records
    duplicates = dataframe.sample(frac=0.05)
    dataframe = pd.concat([dataframe, duplicates], ignore_index=True)

    # Increase purchase_amount by 1000 for 1% of the records
    dataframe.loc[dataframe.sample(frac=0.01).index, 'purchase_amount'] += 1000

    # Set purchase_amount to negative for 1% of the records
    dataframe.loc[dataframe.sample(frac=0.01).index, 'purchase_amount'] *= -1

    # Set purchase_amount to 'unknown' for 1% of the records
    indices_to_change = dataframe.sample(frac=0.01).index
    dataframe.loc[indices_to_change, 'purchase_amount'] = 'unknown'
    # Ensure that the purchase_amount column is of object type if 'unknown' values are introduced
    dataframe['purchase_amount'] = dataframe['purchase_amount'].astype(object)

    na_indices = np.random.choice(dataframe.index, size=int(0.1 * len(dataframe)), replace=False)
    dataframe.loc[na_indices, 'purchase_amount'] = np.nan

    dataframe.loc[np.random.choice(dataframe.index, size=int(0.1 * len(dataframe)), replace=False), 'address'] = np.nan

    # Introduce typos for 5% of the address records
    def introduce_typos(text):
        if pd.isnull(text) or len(text) < 5:  # Jika NaN atau terlalu pendek, tidak berubah
            return text
        char_pos = random.randint(0, len(text) - 1)
        return text[:char_pos] + random.choice('abcdefghijklmnopqrstuvwxyz') + text[char_pos + 1:]
    
    typo_indices = dataframe.sample(frac=0.05).index
    dataframe.loc[typo_indices, 'address'] = dataframe.loc[typo_indices, 'address'].apply(introduce_typos)

    # Concatenating address information from other records to create inconsistencies
    # For a small fraction of records, combine address details from other records
    concat_indices = dataframe.sample(frac=0.05).index
    for idx in concat_indices:
        if len(dataframe) > idx + 1:  # Avoid the last index
            dataframe.at[idx, 'address'] += ' ' + dataframe.at[idx + 1, 'address'].split(' ')[-1]

    return dataframe

dataframe = pd.read_csv('raw_data.csv')
# Assuming df is the DataFrame after the ETL process
# Apply the additional transformations
df = additional_transformations(dataframe)

# Save the transformed DataFrame to a new CSV file
df.to_csv('combined_data.csv', index=False)
load_to_mysql(df, 'new_table', db_params)


print("Additional transformations applied and saved to CSV file.")
print(df)


Additional transformations applied and saved to CSV file.
    id                   name                              email  \
0    1               john doe                                NaN   
1    2             Jane Smith             jane.smith@example.com   
2    3          alice johnson          alice.johnson@example.com   
3    4          michael brown          michael.brown@example.com   
4    5            Emily Davis            emily.davis@example.com   
5    6          daniel garcia          DANIEL.GARCIA@EXAMPLE.COM   
6    7           Laura Wilson           LAURA.WILSON@EXAMPLE.COM   
7    8         Kevin Martinez                                NaN   
8    9      Isabella Anderson      isabella.anderson@example.com   
9   10          Joshua Thomas          joshua.thomas@example.com   
10  11             sophia lee             sophia.lee@example.com   
11  12  christopher hernandez  christopher.hernandez@example.com   
12  13           Jessica King                             

In [6]:
import pandas as pd
import random
from sqlalchemy import create_engine

def clean_data(dataframe):
    # Fill missing emails with a placeholder
    dataframe['email'].fillna('no_email_provided', inplace=True)
    
    # Introduce a placeholder for missing addresses
    dataframe['address'].fillna('No Address Provided', inplace=True)
    
    # Handle missing or 'unknown' purchase_amount
    # Convert 'unknown' to NaN and then fill NaN with the mean of the column
    dataframe['purchase_amount'] = pd.to_numeric(dataframe['purchase_amount'], errors='coerce')
    dataframe['purchase_amount'].fillna(dataframe['purchase_amount'].mean(), inplace=True)
    # dataframe['purchase_amount'].fillna(dataframe['purchase_amount'].median(), inplace=True) #menggunakan nilai median
    # dataframe['purchase_amount'].fillna(dataframe['purchase_amount'].mode()[0], inplace=True) #menggunakan modus
    # dataframe['purchase_amount'].fillna(0, inplace=True) #menggunakan nilai tetap
    # dataframe['purchase_amount'].fillna(method='bfill', inplace=True)  # Backward fill
    # dataframe['purchase_amount'].fillna(method='ffill', inplace=True)  # Forward fill
    # dataframe['purchase_amount'].interpolate(method='linear', inplace=True) #interpolasi
    
    # Correct the date format for purchase_date
    dataframe['purchase_date'] = dataframe['purchase_date'].apply(lambda x: x.replace('/', '-'))
    
    # Normalize the case of names and emails
    dataframe['name'] = dataframe['name'].str.title()
    dataframe['email'] = dataframe['email'].str.lower()
    
    # Remove duplicates
    dataframe.drop_duplicates(inplace=True)
    
    # Reset the index after removing duplicates
    dataframe.reset_index(drop=True, inplace=True)
    
    return dataframe


# Misalkan df adalah DataFrame setelah proses transformasi tambahan
# Membersihkan data dengan fungsi clean_data
df_cleaned = clean_data(df)

# Simpan DataFrame yang telah dibersihkan ke file CSV baru
df_cleaned.to_csv('cleaned_data.csv', index=False)

# Jika Anda ingin memuat data yang telah dibersihkan kembali ke MySQL:
load_to_mysql(df_cleaned, 'clean_table', db_params)

print("Data cleaned and saved to CSV file.")


Data cleaned and saved to CSV file.
